In [1]:
import os
import shutil
from itertools import islice
import requests

import pandas as pd
import matplotlib.pyplot as plt

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Create spark_session
spark = SparkSession.builder.getOrCreate()
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '32g'), ('spark.app.name', 'Spark Updated Conf'), 
                                        ('spark.executor.cores', '32'), ('spark.cores.max', '32'), ('spark.driver.memory','32g')])
spark.sparkContext.getConf().getAll()

from IPython.display import clear_output
clear_output(wait = False)

spark.version

import time
start_time = time.time()

In [2]:
from google.cloud import storage

In [3]:
%%time
df_tweets_master_filtered_keywords = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/jasmeetsingh/df_tweets_master_filtered_keywords_doubleFiltered/')

CPU times: user 6.16 ms, sys: 3.37 ms, total: 9.52 ms
Wall time: 8.31 s


22/12/07 01:36:55 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df_tweets_master_filtered_keywords.count()

3119357

## Which Retweet counts and reply counts to usee

In [5]:
df_retweetCounts = df_tweets_master_filtered_keywords.select([
    df_tweets_master_filtered_keywords.retweet_count.alias("direct_retweet_count"),
    df_tweets_master_filtered_keywords.quoted_status.retweet_count.alias("quoted_status.retweet_count"),
    df_tweets_master_filtered_keywords.retweeted_status.quoted_status.retweet_count.alias("retweeted_status.quoted_status.retweet_count"),
    df_tweets_master_filtered_keywords.retweeted_status.retweet_count.alias("retweeted_status.retweet_count"),
    df_tweets_master_filtered_keywords.retweeted_status.reply_count.alias("retweeted_status.reply_count"),
    df_tweets_master_filtered_keywords.reply_count.alias("reply_count"),
    df_tweets_master_filtered_keywords.tweet_text
]).limit(10000).toPandas()

In [6]:
df_retweetCounts.describe()

,direct_retweet_count,quoted_status.retweet_count,retweeted_status.quoted_status.retweet_count,retweeted_status.retweet_count,retweeted_status.reply_count,reply_count
count,10000.0,780.000000,698.000000,9147.000000,9147.000000,10000.0
mean,0.0,1046.788462,987.094556,3955.399038,1199.493932,0.0
std,0.0,3220.334563,2758.472976,9681.955323,2887.967430,0.0
min,0.0,0.000000,0.000000,1.000000,0.000000,0.0
25%,0.0,15.750000,16.250000,18.000000,2.000000,0.0
50%,0.0,157.500000,177.500000,263.000000,40.000000,0.0
75%,0.0,455.500000,453.000000,3500.500000,486.000000,0.0
max,0.0,50009.000000,23650.000000,100947.000000,27268.000000,0.0


In [7]:
# Best will be retweeted_status.retweet_count

In [8]:
df_original_tweets = df_tweets_master_filtered_keywords.filter("retweeted_status.retweet_count is not null") 
df_original_tweets.count()

2800366

In [9]:
# df_retweetCounts.filter("direct_retweet_count is not null and quoted_status.retweet_count is not null \
#                         and retweeted_status.quoted_status.retweet_count is not null and retweeted_status.retweet_count is not null").limit(20).toPandas()

## Which geographical data column to use

In [10]:
df_tweets_master_filtered_keywords.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [11]:
df_geodata = df_tweets_master_filtered_keywords.select([
    df_tweets_master_filtered_keywords.coordinates.alias("direct_coordinates"),
    df_tweets_master_filtered_keywords.geo.coordinates.alias("geo.coordinates"),
    df_tweets_master_filtered_keywords.place.bounding_box.coordinates.alias("place.bounding_box.coordinates"),
    df_tweets_master_filtered_keywords.place.country_code.alias("place.country_code"),
    df_tweets_master_filtered_keywords.place.country.alias("place.country"),
    df_tweets_master_filtered_keywords.place['name'].alias("place.name"),
    df_tweets_master_filtered_keywords.place.full_name.alias("place.full_name"),
    df_tweets_master_filtered_keywords.user.location.alias("user.location"),
    df_tweets_master_filtered_keywords.tweet_text,
    df_tweets_master_filtered_keywords.user.withheld_in_countries
]).limit(30000).toPandas()

In [12]:
df_geodata.describe()

,direct_coordinates,geo.coordinates,place.bounding_box.coordinates,place.country_code,place.country,place.name,place.full_name,user.location,tweet_text,user.withheld_in_countries
count,12,12,51,51,51,51,51,18605,30000,30000
unique,9,9,45,10,10,44,45,9254,12973,1
top,"([78.57329071, 13.36094689], Point)","[13.36094689, 78.57329071]","[[[78.564374, 13.356813], [78.564374, 13.37734...",US,United States,Punganuru,"Punganuru, India",United States,The so-called “Supreme Court” just ruled 6-3 t...,[]
freq,4,4,4,31,31,4,4,414,1341,30000


In [13]:
# df_geodata['user.withheld_in_countries'].isnull().count()

In [14]:
df_tweets_master_filtered_keywords.filter("user.withheld_in_countries is not NULL").count()

3119357

In [15]:
df_tweets_master_filtered_keywords.limit(5).toPandas()

,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,...,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_copyright,withheld_in_countries,important
0,None,Mon Aug 08 20:25:07 +0000 2022,None,"([], None, [], [], [(44513878, 44513878, [3, 1...",None,None,0,False,low,None,...,"(None, Mon Aug 08 15:17:00 +0000 2022, None, (...","<a href=""http://twitter.com/download/iphone"" r...",rt @texastribune: school choice critics worry ...,1659990307605,False,School choice critics worry that the policy co...,"(False, Fri Jul 24 12:18:09 +0000 2009, True, ...",None,None,1
1,None,Mon Aug 08 20:25:07 +0000 2022,None,"([], None, [], [(cincinnati.com/story/news/202...",None,None,0,False,low,None,...,"(None, Fri Aug 05 12:34:28 +0000 2022, None, (...","<a href=""https://mobile.twitter.com"" rel=""nofo...",rt @cweiser: design lab: grade school students...,1659990307586,False,Design LAB: Grade school students learn to be ...,"(False, Fri Dec 02 06:14:35 +0000 2011, False,...",None,None,1
2,None,Mon Aug 08 20:25:20 +0000 2022,None,"([], None, [], [], [(1124414849331662850, 1124...",None,None,0,False,low,None,...,"(None, Tue Aug 02 13:16:19 +0000 2022, None, (...","<a href=""http://twitter.com/download/iphone"" r...",rt @csetgeorgetown: if the united states aims ...,1659990320324,False,If the United States aims to outcompete other ...,"(False, Tue Feb 17 01:38:51 +0000 2009, True, ...",None,None,1
3,None,Mon Aug 08 20:25:22 +0000 2022,None,"([], None, [], [], [(2832807293, 2832807293, [...",None,None,0,False,low,None,...,"(None, Mon Aug 08 19:52:38 +0000 2022, [0, 140...","<a href=""http://twitter.com/download/iphone"" r...",rt @hiitaylorblake: i’ve had a few teachers as...,1659990322579,False,I’ve had a few teachers ask me for personalize...,"(False, Fri Dec 03 16:42:35 +0000 2010, True, ...",None,None,1
4,None,Mon Aug 08 20:25:25 +0000 2022,None,"([], None, [], [], [(1276140426, 1276140426, [...",None,None,0,False,low,None,...,"(None, Mon Aug 08 18:02:16 +0000 2022, None, (...","<a href=""http://twitter.com/download/iphone"" r...",rt @deangeliscorey: breaking: a coalition of c...,1659990325887,False,BREAKING: A coalition of charter schools filed...,"(False, Fri Jan 23 02:54:05 +0000 2015, True, ...",None,None,1


In [16]:
df_tweets_master_filtered_keywords.limit(5).toPandas().columns

Index(['coordinates', 'created_at', 'display_text_range', 'entities',
       'extended_entities', 'extended_tweet', 'favorite_count', 'favorited',
       'filter_level', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'lang', 'place', 'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'quoted_text', 'reply_count', 'retweet_count', 'retweeted',
       'retweeted_from', 'retweeted_status', 'source', 'text', 'timestamp_ms',
       'truncated', 'tweet_text', 'user', 'withheld_copyright',
       'withheld_in_countries', 'important'],
      dtype='object')

In [17]:
df_final_for_analysis = df_tweets_master_filtered_keywords.select([
    df_tweets_master_filtered_keywords.created_at,
    df_tweets_master_filtered_keywords.id,
    df_tweets_master_filtered_keywords.geo.coordinates.alias("geo_coordinates"),
    df_tweets_master_filtered_keywords.user['name'].alias("user_name"), 
    df_tweets_master_filtered_keywords.user.followers_count.alias("followers_count"), 
    df_tweets_master_filtered_keywords.user.verified.alias("verified_user"),
    df_tweets_master_filtered_keywords.user.location.alias("user_location"),
    df_tweets_master_filtered_keywords.user.description.alias("user_description"),
    df_tweets_master_filtered_keywords.retweeted_status.reply_count.alias("reply_count"),
    df_tweets_master_filtered_keywords.retweeted_status.retweet_count.alias("retweet_count"),
    df_tweets_master_filtered_keywords.retweeted_status.alias("retweeted_status"),
    df_tweets_master_filtered_keywords.tweet_text,
    df_tweets_master_filtered_keywords.text,
])

df_final_for_analysis.limit(5).toPandas()

,created_at,id,geo_coordinates,user_name,followers_count,verified_user,user_location,user_description,reply_count,retweet_count,retweeted_status,tweet_text,text
0,Mon Aug 08 20:25:07 +0000 2022,1556738297606569994,None,Nebula 🛹☮️,9247,False,ShitholeImmigrant,Proud Liberal #Betocrat #BetoForGovernor #Vote...,20,105,"(None, Mon Aug 08 15:17:00 +0000 2022, None, (...",School choice critics worry that the policy co...,rt @texastribune: school choice critics worry ...
1,Mon Aug 08 20:25:07 +0000 2022,1556738297526902787,None,Sydney Rey Franklin,457,False,"Cincinnati, OH",@enquirer real estate enterprise reporter // F...,0,1,"(None, Fri Aug 05 12:34:28 +0000 2022, None, (...",Design LAB: Grade school students learn to be ...,rt @cweiser: design lab: grade school students...
2,Mon Aug 08 20:25:20 +0000 2022,1556738350953861121,None,Kate F,661,False,California,An aware & concerned American.,0,6,"(None, Tue Aug 02 13:16:19 +0000 2022, None, (...",If the United States aims to outcompete other ...,rt @csetgeorgetown: if the united states aims ...
3,Mon Aug 08 20:25:22 +0000 2022,1556738360412086280,None,David Engelson,692,False,"Staten Island, NY","Cogito, ergo sum. Wife : @CeliaEngelson. Daugh...",113,352,"(None, Mon Aug 08 19:52:38 +0000 2022, [0, 140...",I’ve had a few teachers ask me for personalize...,rt @hiitaylorblake: i’ve had a few teachers as...
4,Mon Aug 08 20:25:25 +0000 2022,1556738374286934018,None,Kathryn Moore,186,False,None,None,24,364,"(None, Mon Aug 08 18:02:16 +0000 2022, None, (...",BREAKING: A coalition of charter schools filed...,rt @deangeliscorey: breaking: a coalition of c...


In [28]:
df_nulls = df_final_for_analysis.select([
                F.count(
                    F.when(df_final_for_analysis[column].isNull(), column)
                ).alias(column) for column in df_final_for_analysis.columns
            ]).toPandas()

# display.max_columns : int

In [29]:
df_nulls

,created_at,id,geo_coordinates,user_name,followers_count,verified_user,user_location,user_description,reply_count,retweet_count,retweeted_status,tweet_text,text
0,0,0,3117914,0,0,0,1187681,574797,318991,318991,318991,0,0


In [30]:
%%time 
df_final_for_analysis.write.mode("overwrite").parquet("gs://msca-bdp-students-bucket/shared_data/jasmeetsingh/df_final_for_analysis/")

CPU times: user 130 ms, sys: 65.8 ms, total: 196 ms
Wall time: 1min 53s


In [21]:
print("--- Total Time to run the notebook: %s seconds ---" % (time.time() - start_time))

--- Total Time to run the notebook: 666.7253811359406 seconds ---
